In [6]:
import numpy as np
from scipy.linalg import block_diag


In [16]:
# Trend
class dlm_trend(object):
    
    def __init__(self,order,a = None,R = 0.01):
        
        if a is None:
            a = [10]*order

        
        self.order = order
        # F
        Ftrend = np.mat(np.zeros(self.order).tolist())
        Ftrend[0] = 1
        Ftrend.shape = (order,1)
        self.Ftrend = np.matrix(Ftrend)

        # G 
        subG = np.identity(self.order-1)
        G = np.zeros((self.order,self.order))
        G[:order-1,1:] = subG
        np.fill_diagonal(G,1) 
        self.Gtrend = np.matrix(G)

        # prior: a, R
        self.a = np.array(a)
        self.R = np.array(R)
        
# Regressor
class dlm_reg(object):
    
    def __init__(self,q, a = None, R = 0.01):
        self.q = q
        
        
        if a is None:
            a = [10]*self.q
        
        Freg = np.mat(np.zeros(self.q))
        Freg.shape = (self.q,1)
        self.Freg = Freg
        

        Greg = np.identity(self.q)
        self.Greg = Greg
        
        # prior: a, R
        self.a = np.array(a)
        self.R = np.array(R)
        
# seasonal 
class dlm_season(object):
    
    
    def __init__(self,p , rseas = [1,3,4],a = None, R = 0.01):
        
        
        
        
        
        pseas = len(rseas)
        nseas = 2*pseas
        Fseas = np.mat(np.array([1,0]*pseas))
        Fseas.shape = (2*pseas,1)
        Gseas = np.mat(np.zeros((nseas,nseas)))
        
        for j in range(pseas):
            i = j + 1 
            c = np.cos(2*np.pi*rseas[j]/p)
            s = np.sin(2*np.pi*rseas[j]/p)
            index = list(range(2*i-1,2*i+1))

            Gseas[index[0]-1:index[1],index[0]-1:index[1]] = np.mat(np.array([[c,s],[-s,c]]))


        self.Gseas = Gseas
        self.Fseas = Fseas
        
        # prior: a, R

        if a is None:
            a = [10]*nseas
            
        self.a = np.array(a)
        R_T = np.identity(nseas)
        np.fill_diagonal(R_T,R)
        self.R = R_T
        

 

In [35]:
class dlm_mod(object):
    
    def __init__(self, dlm_trend = None, dlm_reg = None, dlm_seas = None, nu = 6, S = np.power(0.15,2)):
    
        F = np.concatenate((dlm_trend.Ftrend, dlm_reg.Freg, dlm_seas.Fseas))
        G = np.mat(block_diag(dlm_trend.Gtrend,dlm_reg.Greg,dlm_seas.Gseas))
        R = np.mat(block_diag(dlm_trend.R,dlm_reg.R,dlm_seas.R))
        
        a = np.mat(np.concatenate([dlm_trend.a.tolist()] + [dlm_reg.a.tolist()] + [dlm_seas.a.tolist()]))
        a.shape = (a.shape[1],1)
        
        self.F = F
        self.G = G
        self.R = R
        self.a = a
        self.nu = nu
        self.S = S

In [36]:
mod_trend = dlm_trend(1, R = 0.09)
mod_reg = dlm_reg(q = 3)
mod_seas = dlm_season(52,[1,3],R = 0.0067)

In [37]:
dlm_mod(mod_trend,mod_reg,mod_seas).F

matrix([[ 1.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 0.]])

In [38]:
dlm_mod(mod_trend,mod_reg,mod_seas).G

matrix([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.99270887,
          0.12053668,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        , -0.12053668,
          0.99270887,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.93501624,  0.35460489],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        , -0.35460489,  0.93501624]])

In [39]:
dlm_mod(mod_trend,mod_reg,mod_seas).R

matrix([[ 0.09  ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.01  ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.0067,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.0067,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.0067,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.0067]])

In [40]:
dlm_mod(mod_trend,mod_reg,mod_seas).a

matrix([[10],
        [10],
        [10],
        [10],
        [10],
        [10],
        [10],
        [10]])